In [1]:
import os
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall, context_entity_recall, answer_similarity, answer_correctness
from constants import EVAL_ANSWERS, EVAL_QUESTIONS
from rag import caller_with_context

In [2]:
from datasets import Dataset

In [3]:
queries = EVAL_QUESTIONS
ground_truths = EVAL_ANSWERS

In [13]:
sess_id = "X123"
results = []
contexts = []

for query in queries:
    result = caller_with_context(query, sess_id)
    print(result)
    results.append(result['answer'])
    sources = result["context"]
    contents = []
    for i in range(len(sources)):
        contents.append(sources[i][0].page_content)
    contexts.append(contents)

d = {
    "question": queries,
    "answer": results,
    "contexts": contexts,
    "ground_truth": ground_truths
}

dataset = Dataset.from_dict(d)
score = evaluate(dataset, metrics=[faithfulness, answer_relevancy, context_precision, context_recall, context_entity_recall, answer_similarity, answer_correctness, harmfulness])
score_df = score.to_pandas()
score_df.to_csv("EvaluationScores.csv", encoding="utf-8", index=False)

score_df[['faithfulness','answer_relevancy', 'context_precision', 'context_recall',
       'context_entity_recall', 'answer_similarity', 'answer_correctness',
       'harmfulness']].mean(axis=0)




{'result': 'ketentuan_terkait', 'question': 'Apa batas waktu yang diberikan untuk persetujuan atau penolakan atas permohonan izin usaha setelah dokumen permohonan diterima secara lengkap?', 'history': []}


Error in RootListenersTracer.on_chain_end callback: KeyError('output')


{'result': 'ketentuan_terkait', 'question': 'Berapa lama waktu yang diberikan bagi bank yang telah mendapat izin usaha dari Gubernur Bank Indonesia untuk memulai kegiatan usaha perbankan?', 'history': []}


Error in RootListenersTracer.on_chain_end callback: KeyError('output')


{'result': 'ketentuan_terkait', 'question': 'Siapa yang dapat mendirikan dan memiliki Bank Perantara?', 'history': []}


Error in RootListenersTracer.on_chain_end callback: KeyError('output')


KeyboardInterrupt: 

In [12]:
sources[1][0].page_content

'Nomor Ketentuan: 11/1/PBI/2009\nKetentuan: Pasal 10 \nmemiliki Isi ketentuan sebagai berikut: Permohonan untuk mendapatkan izin usaha sebagaimana dimaksud dalam Pasal 4 ayat (2) huruf b diajukan oleh pihak yang telah mendapat persetujuan prinsip kepada Gubernur Bank Indonesia, disertai dengan:\na. akta pendirian badan hukum, yang memuat Anggaran Dasar yang telah disahkan oleh instansi berwenang;\nb. data kepemilikan sebagaimana dimaksud dalam Pasal 7 ayat (1) huruf b yang masing-masing disertai dengan dokumen sebagaimana dimaksud dalam Pasal 7 ayat (2), dalam hal terjadi perubahan kepemilikan;\nc. daftar susunan Dewan Komisaris dan Direksi, disertai dengan:\n1. contoh tanda tangan dan paraf;\n2. identitas dan dokumen sebagaimana dimaksud dalam Pasal 7 ayat (1) huruf c, dalam hal terjadi perubahan; dan\n3. fotokopi Kartu Izin Tinggal Terbatas (KITAS) atau Kartu Izin Tinggal Tetap (KITAP) dan fotokopi surat izin bekerja dari instansi berwenang,bagi warga negara asing:'